# AML Project - Anomaly Segmentation

## Notebook setup

In [ ]:
%cd /content
!rm -r AnomalySegmentation_CourseProjectBaseCode
!git clone https://github.com/AML-project-AnomalySegmentation/AnomalySegmentation_CourseProjectBaseCode.git


In [ ]:
!pip3 install visdom
!pip3 install ood_metrics



### Download Cityscape Dataset


In [ ]:
%cd AnomalySegmentation_CourseProjectBaseCode/dataset
!gdown 1bCoKg9_Z1tVZS66c_JrWuVeKxsT6QzDz
!jar xvf "Cityscapes.zip"


## Steps

In [2]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval

/content/AnomalySegmentation_CourseProjectBaseCode/eval


### Step 2A

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
DISCRIMINANTS = ["msp", "maxlogit", "maxentropy"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  for disc in (DISCRIMINANTS):
    print(f"Dataset: {d} Discriminant: {disc}")
    !python evalAnomaly.py  --input {path} --discriminant {disc}

In [ ]:
DISCRIMINANTS = ["msp", "maxlogit", "maxentropy"]
for d in DISCRIMINANTS:
   !python eval_iou.py --discriminant {d}

In [ ]:
TEMPERATURE = [0.5, 0.75, 1.1]
for d in TEMPERATURE:
   !python eval_iou.py --temperature {d}

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
path = f"'Validation_Dataset/'"
print(f"Dataset: {d}")
!python optimalTemperature.py  --input {path}

### Step 2B

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
TEMPERATURES = [1,0.5,0.75,1.1]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  for t in (TEMPERATURES):
    print(f"Dataset: {d} Temeperature: {t}")
    !python evalAnomaly.py  --input {path} --temperature {t}


## Step 3

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
MODELS = ["ErfNet", "ENet", "BiseNet"]
for m in MODELS:
  for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d}")
    !python VoidEvalAnomaly.py  --input {path} --model {m}

In [ ]:
MODELS = ["ErfNet", "ENet", "BiseNet"]
for m in MODELS:
  !python void_eval_iou.py --model {m}



## Step 4



### Download CamVid Dataset



In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/dataset
!gdown 12Q3ayeWZnl8J8bAGermmLKr3G89jN0Sa
!jar xvf "CamVid.zip"


## Test Dataset


In [6]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --datadir "../dataset/CamVid"

/content/AnomalySegmentation_CourseProjectBaseCode/train
../dataset/CamVid/train/ ../dataset/CamVid/train_labels/
../dataset/CamVid/val/ ../dataset/CamVid/val_labels/
369
